In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from catboost import Pool, CatBoostRegressor, cv, MetricVisualizer
# from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score, make_scorer
# from sklearn.model_selection import KFold

# from sklearn.feature_selection import SelectKBest, f_regression

In [2]:
np.random.seed(7)

# data load

In [3]:
base = os.getcwd()
data_path = os.path.join(base, 'data')
submit_path = os.path.join(base, 'submit')
model_path = os.path.join(base, 'model')

def load_data(name):
    return np.load(os.path.join(data_path, f"{name}.npy"))

def reshape(data):
    return data.reshape(data.shape[0] * 40 * 40, data.shape[-1])

if not os.path.isdir(model_path):
    os.mkdir(model_path)

In [4]:
data = reshape(load_data('dl_train'))

# seperate dataset

In [5]:
X = data[:, :-1]
Y = data[:,  -1].reshape(data.shape[0], 1)
data = range(data.shape[0])
print(X.shape, Y.shape)

(121561600, 14) (121561600, 1)


# CatBoostRegressor

In [7]:
clf = CatBoostRegressor(iterations=200, learning_rate=0.1, \
                        depth=4, l2_leaf_reg=20, \
                        bootstrap_type='Bernoulli', subsample=0.6, \
                        eval_metric='RMSE', metric_period=10, \
                        od_type='Iter', od_wait=45, random_seed=17,\
                        allow_writing_files=False)

In [ ]:
clf.fit(X, Y, \
        cat_features=[],\
        use_best_model=True, \
        plot=True, verbose=True)

# 제출

In [ ]:
def submit(clf, name, preprocess=None):
    x_test = reshape(load_data('test'))
    
    pred = clf.predict(x_test)

    submission = pd.read_csv(os.path.join(data_path, 'sample_submission.csv'))
    submission.iloc[:, 1:] = pred.reshape(-1, 1600)

    submission.to_csv(os.path.join(submit_path, f'{name}.csv'), index=False)

In [16]:
submit(clf, 'catboost_all_Bernoulli_RMSE')

- https://dacon.io/competitions/official/235591/mysubmission/
- D:\인공지능_공모전\github\submit